In [2]:
import json
import pandas as pd
import os

import pyspark.sql.functions as F
from pyspark.sql.types import StringType

out_path = 's3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/sampling/watched_time_for_collapse/'
df = spark.read.parquet(f'{out_path}/quarter_data/')

In [3]:
df.count()

61517745

In [4]:
@F.udf(returnType=StringType())
def nccs(segments):
    if segments is not None:
        for x in segments.split('|'):
            if x.startswith('NCCS_'):
                return x
    return 'other'

@F.udf(returnType=StringType())
def gender(segments):
    if segments is not None:
        for x in segments.split('|'):
            if x.startswith('FMD00') or '_FEMALE_' in x:
                return 'f'
            if x.startswith('MMD00') or '_MALE_' in x:
                return 'm'
    return 'other'

@F.udf(returnType=StringType())
def age(segments):
    dc = {
        'FB_MALE_35-44',
        'FB_MALE_45-54',
        'FB_MALE_55-64',
        'FB_MALE_65PLUS',
        'FB_FEMALE_35-44',
        'FB_FEMALE_45-54',
        'FB_FEMALE_55-64',
        'FB_FEMALE_65PLUS',
        'FB_BARC_FEMALE_31-40',
        'FB_BARC_FEMALE_41-50',
        'FB_BARC_FEMALE_51+',
        'FB_BARC_MALE_31-40',
        'FB_BARC_MALE_41-50',
        'FB_BARC_MALE_51+',
        'EMAIL_MALE_35-44',
        'EMAIL_MALE_45-54',
        'EMAIL_MALE_55-64',
        'EMAIL_MALE_65PLUS',
        'EMAIL_FEMALE_35-44',
        'EMAIL_FEMALE_45-54',
        'EMAIL_FEMALE_55-64',
        'EMAIL_FEMALE_65PLUS',
        'EMAIl_BARC_FEMALE_31-40',
        'EMAIl_BARC_FEMALE_41-50',
        'EMAIl_BARC_FEMALE_51+',
        'EMAIl_BARC_MALE_31-40',
        'EMAIl_BARC_MALE_41-50',
        'EMAIl_BARC_MALE_51+',
        'PHONE_MALE_35-44',
        'PHONE_MALE_45-54',
        'PHONE_MALE_55-64',
        'PHONE_MALE_65+',
        'PHONE_FEMALE_35-44',
        'PHONE_FEMALE_45-54',
        'PHONE_FEMALE_55-64',
        'PHONE_FEMALE_65+',
        'PHONE_MALE_TV_31-40',
        'PHONE_MALE_TV_41-50',
        'PHONE_MALE_TV_51-60',
        'PHONE_MALE_TV_60+',
        'PHONE_FEMALE_TV_31-40',
        'PHONE_FEMALE_TV_41-50',
        'PHONE_FEMALE_TV_51-60',
        'PHONE_FEMALE_TV_60+',
        'PHONE_BARC_FEMALE_31-40',
        'PHONE_BARC_FEMALE_41-50',
        'PHONE_BARC_FEMALE_51+',
        'PHONE_BARC_MALE_31-40',
        'PHONE_BARC_MALE_41-50',
        'PHONE_BARC_MALE_51+',
        'FMD009V0053599SRMLDESTADS',
        'MMD009V0053599SRMLDESTADS',
        'FMD009V0053599HIGHSRMLDESTADS',
        'MMD009V0053599HIGHSRMLDESTADS',
    }
    if segments is not None:
        for x in segments.split('|'):
            if x in dc:
                return '30+'
    return 'other'

@F.udf(returnType=StringType())
def device(segments):
    if segments is not None:
        dc = {'A_40990869': '25K+', 'A_21231588': '25K+'}
        for x in segments.split('|'):
            if x in dc:
                return dc[x]
    return 'other'

pdf = df.groupby(
    'cd', 'is_cricket',
    'country', 'language', 'platform', 'city', 'state',
    nccs('segments').alias('nccs'), 
    device('segments').alias('device'),
    gender('segments').alias('gender'),
    age('segments').alias('age'),
).agg(F.sum('watch_time').alias('watch_time'),
      F.sum('reach').alias('reach'))

pdf.repartition(8).write.mode('overwrite').parquet(f'{out_path}quarter_data_v2')

[Stage 4:========================================================>(68 + 1) / 69]23/03/11 03:06:46 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


In [5]:
pdf

cd,is_cricket,country,language,platform,city,state,nccs,device,gender,age,watch_time,reach
2021-10-24,false,in,hindi,android,puri,or,NCCS_A,other,f,30+,3309905.0,1159
2021-10-24,false,in,hindi,android,na,na,NCCS_C_D,other,m,30+,31.0,2
2021-10-24,false,us,hindi,ios,nashville,tn,other,25K+,f,other,5.466714859008789,1
2021-10-24,false,in,kannada,android,shimoga,ka,NCCS_A,other,m,30+,39311.0,30
2021-10-24,false,in,hindi,android,limbdi,gj,NCCS_A,other,f,30+,13732.0,1
2021-10-24,false,in,hindi,android,virudhunagar,tn,NCCS_C_D,other,m,30+,183.0,5
2021-10-24,false,in,english,android,surat,gj,NCCS_B,other,f,other,85098.0,112
2021-10-24,false,in,english,web,kolhapur,mh,NCCS_A,other,m,other,39698.94966709614,17
2021-10-24,false,in,english,android,shimla,hp,other,other,m,30+,17004.0,35
2021-10-24,false,in,hindi,android,kharar,pb,NCCS_A,other,f,other,281577.0,137


In [7]:
!aws s3 sync s3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/sampling/watched_time_for_collapse/quarter_data_v2/ quarter_data_v2

download: s3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/sampling/watched_time_for_collapse/quarter_data_v2/_SUCCESS to quarter_data_v2/_SUCCESS
download: s3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/sampling/watched_time_for_collapse/quarter_data_v2/part-00000-593766a8-e9ff-418f-921b-b18263192cf7-c000.snappy.parquet to quarter_data_v2/part-00000-593766a8-e9ff-418f-921b-b18263192cf7-c000.snappy.parquet
download: s3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/sampling/watched_time_for_collapse/quarter_data_v2/part-00002-593766a8-e9ff-418f-921b-b18263192cf7-c000.snappy.parquet to quarter_data_v2/part-00002-593766a8-e9ff-418f-921b-b18263192cf7-c000.snappy.parquet
download: s3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/sampling/watched_time_for_collapse/quarter_data_v2/part-00003-593766a8-e9ff-418f-921b-b18263192cf7-c000.snappy.parquet to quarter_data_v2/part-00003-593766a8-e9ff-4

In [9]:
import pandas as pd
import numpy as np

path = 's3://adtech-ml-perf-ads-us-east-1-prod-v1/live_inventory_forecasting/data/sampling/watched_time_for_collapse/quarter_data_v2/'
basic = ['platform', 'device', 'language', 'city', 'state', 'age', 'gender']
df = pd.read_parquet('quarter_data_v2')
df

,cd,is_cricket,country,language,platform,city,state,nccs,device,gender,age,watch_time,reach
0,2022-10-27,False,in,japanese,ios,gurgaon,hr,NCCS_A,25K+,f,30+,54.250981,1
1,2022-10-26,False,in,hindi,ios,nakodar,pb,other,other,f,other,1996.767343,1
2,2021-10-27,False,in,telugu,android,bandarlanka,ap,other,other,m,30+,14795.000000,11
3,2021-11-01,True,in,english,android,khekra,up,NCCS_A,other,m,other,65.000000,1
4,2022-11-05,False,in,hindi,web,davangere,ka,other,other,other,other,5846.049805,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10583391,2021-10-19,False,in,hindi,web,pachora,mh,NCCS_A,other,m,other,349.130005,1
10583392,2022-11-08,False,in,hindi,android,junagadh,gj,other,25K+,f,other,88417.000000,32
10583393,2022-11-15,False,in,hindi,ios,panipat,hr,NCCS_A,other,f,30+,11.453921,1
10583394,2021-10-22,False,in,malayalam,android,jodhpur,rj,other,other,m,other,1.000000,1


In [104]:
translation = {
    'city': {
        'mumbai':'mumbai',
        'delhi':'delhi',
        'bangalore':'bangalore',
        'hyderabad':'hyderabad',
        'ahmedabad':'ahmedabad',
        'chennai':'chennai',
        'kolkata':'kolkata',
        'surat':'surat',
        'pune':'pune',
        'jaipur':'jaipur',
        'lucknow':'lucknow',
        'kanpur':'kanpur',
        'nagpur':'nagpur',
        'indore':'indore',
        'bhopal':'bhopal',
        'visakhapatnam':'visakhapatnam',
        'patna':'patna',
        'vadodara':'vadodara',
        'ludhiana':'ludhiana',
        'agra':'agra',
        'nashik':'nashik',
        'varanasi':'varanasi',
        'dhanbad':'dhanbad',
        'amritsar':'amritsar',
        'allahabad':'allahabad',
        'ranchi':'ranchi',
        'gwalior':'gwalior',
        'coimbatore':'coimbatore',
        'vijayawada':'vijayawada',
        'jodhpur':'jodhpur',
        'madurai':'madurai',
        'raipur':'raipur',
        'kochi':'kochi',
        'chandigarh':'chandigarh',
        'guwahati':'guwahati',
        'other': 'other',
    },
    'platform': {
        'android': 'mobile',
        'ios': 'mobile',
        'other': None,
    },
    'language': {
        'english': 'english',
        'hindi': 'hindi',
        'other': None,
    },
    'gender': {
        'm': 'male',
        'other': 'female',
    },
    'state': {
        'up': 'Uttar Pradesh + Uttarakhand',
        'ut': 'Uttar Pradesh + Uttarakhand',
        'mh': 'Maharashtra',
        'br': 'Bihar + Jharkhand',
        'jh': 'Bihar + Jharkhand',
        'wb': 'West Bengal',
        'ct': 'Chattisgarh',
        'tn': 'Tamil Nadu + Pondicherry',
        'ka': 'Karnataka',
        'gj': 'Gujarat + Goa',
        'ap': 'Andhra Pradesh + Telangana',
        'tg': 'Andhra Pradesh + Telangana',
        'or': 'Odisha',
        'kl': 'Kerala',
        'pb': 'Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir',
        'hr': 'Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir',
        'hp': 'Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir',
        'jk': 'Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir',
        'ar': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'as': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'mn': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'ml': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'mz': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'nl': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'sk': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'tr': 'Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura',
        'rj': 'Rajasthan',
        'mp': 'Madhya Pradesh',
        'other': 'other',
    },
}
possible_geo = set([
    ('mumbai','Maharashtra'),
    ('delhi','Uttar Pradesh + Uttarakhand'),
    ('delhi','Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir'),
    ('delhi','other'), # last one would be in dict
    ('bangalore','Karnataka'),
    ('hyderabad','Andhra Pradesh + Telangana'),
    ('calcutta','West Bengal'),
    ('kolkata','West Bengal'),
    ('chennai','Tamil Nadu + Pondicherry'),
    ('ahmedabad','Gujarat + Goa'),
    ('surat','Gujarat + Goa'),
    ('pune','Maharashtra'),
    ('jaipur','Rajasthan'),
    ('lucknow','Uttar Pradesh + Uttarakhand'),
    ('kanpur','Uttar Pradesh + Uttarakhand'),
    ('nagpur','Maharashtra'),
    ('indore','Madhya Pradesh'),
    ('bhopal','Madhya Pradesh'),
    ('visakhapatnam','Gujarat + Goa'),
    ('patna','Bihar + Jharkhand'),
    ('vadodara','Gujarat + Goa'),
    ('ludhiana','Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir'),
    ('agra','Uttar Pradesh + Uttarakhand'),
    ('nashik','Maharashtra'),
    ('varanasi','Uttar Pradesh + Uttarakhand'),
    ('dhanbad','Bihar + Jharkhand'),
    ('amritsar','Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir'),
    ('allahabad','Uttar Pradesh + Uttarakhand'),
    ('ranchi','Bihar + Jharkhand'),
    ('gwalior','Madhya Pradesh'),
    ('coimbatore','Tamil Nadu + Pondicherry'),
    ('vijayawada','Andhra Pradesh + Telangana'),
    ('jodhpur','Rajasthan'),
    ('madurai','Tamil Nadu + Pondicherry'),
    ('raipur','Chattisgarh'),
    ('kochi','Kerala'),
    ('chandigarh','Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir'),
    ('guwahati','Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura'),
    ('other','Uttar Pradesh + Uttarakhand'),
    ('other','Maharashtra'),
    ('other','Bihar + Jharkhand'),
    ('other','West Bengal'),
    ('other','Chattisgarh'),
    ('other','Tamil Nadu + Pondicherry'),
    ('other','Karnataka'),
    ('other','Gujarat + Goa'),
    ('other','Andhra Pradesh + Telangana'),
    ('other','Odisha'),
    ('other','Kerala'),
    ('other','Haryana + Punjab + Himachal Pradesh + Jammu and Kashmir'),
    ('other','Arunachal Pradesh + Assam +Manipur + Meghalaya+ Mizoram + Nagaland +Sikkim +Tripura'),
    ('other','Rajasthan'),
    ('other','Madhya Pradesh'),
    ('other','other'),
])
most_possible = dict(possible_geo)

def fix_state(row):
    city, state = row
    if (city, state) in possible_geo:
        return state
    return most_possible.get(city, 'other')

def merge(df):
    df2 = df.copy()
    for col, mp in translation.items(): # except state
        df2[col] = df[col].map(lambda x: mp.get(x, mp['other']))
    df2 = df2.dropna() # remove invalid rows
    df2['state'] = df2[['city', 'state']].apply(fix_state, axis=1)
    return df2

df2 = merge(df)
df2

,cd,is_cricket,country,language,platform,city,state,nccs,device,gender,age,watch_time,reach
1,2022-10-26,False,in,hindi,mobile,other,Haryana + Punjab + Himachal Pradesh + Jammu an...,other,other,female,other,1.996767e+03,1
3,2021-11-01,True,in,english,mobile,other,Uttar Pradesh + Uttarakhand,NCCS_A,other,male,other,6.500000e+01,1
5,2021-11-10,False,in,english,mobile,other,Haryana + Punjab + Himachal Pradesh + Jammu an...,NCCS_A,25K+,male,other,2.032021e+03,3
8,2022-10-18,True,in,english,mobile,other,Rajasthan,NCCS_A,other,male,30+,4.288000e+03,8
10,2022-10-19,False,in,english,mobile,other,Kerala,NCCS_C_D,25K+,female,other,5.000000e+00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10583388,2022-11-11,False,in,hindi,mobile,other,Haryana + Punjab + Himachal Pradesh + Jammu an...,NCCS_A,other,male,30+,2.835000e+03,1
10583389,2022-11-09,True,in,hindi,mobile,patna,Bihar + Jharkhand,other,other,female,other,5.124390e+06,6998
10583392,2022-11-08,False,in,hindi,mobile,other,Gujarat + Goa,other,25K+,female,other,8.841700e+04,32
10583393,2022-11-15,False,in,hindi,mobile,other,Haryana + Punjab + Himachal Pradesh + Jammu an...,NCCS_A,other,female,30+,1.145392e+01,1


In [105]:
df2.nunique()

cd                 59
is_cricket          2
country           105
language            2
platform            1
city               36
state              16
nccs                4
device              2
gender              2
age                 2
watch_time    1804444
reach           18715
dtype: int64

In [106]:
df3 = df2[(df2.cd.map(str) > '2022-01-01') & df2.is_cricket].groupby(basic).sum().reset_index().sort_values('reach', ascending=False)
df3

,platform,device,language,city,state,age,gender,watch_time,reach
448,mobile,other,english,delhi,other,other,male,1.550403e+10,6174989
652,mobile,other,hindi,delhi,other,other,male,7.008821e+09,5101860
504,mobile,other,english,mumbai,Maharashtra,other,male,9.854801e+09,4443651
580,mobile,other,english,patna,Bihar + Jharkhand,other,male,9.845727e+09,4274878
492,mobile,other,english,lucknow,Uttar Pradesh + Uttarakhand,other,male,9.710980e+09,4107469
...,...,...,...,...,...,...,...,...,...
48,mobile,25K+,english,gwalior,Madhya Pradesh,30+,female,2.980132e+02,2
458,mobile,other,english,gwalior,Madhya Pradesh,30+,male,1.614665e+03,2
457,mobile,other,english,gwalior,Madhya Pradesh,30+,female,1.892675e+03,2
661,mobile,other,hindi,gwalior,Madhya Pradesh,30+,female,2.839527e+02,1


In [102]:
(df3[['reach', 'watch_time']].sort_values('reach', ascending=False).cumsum() / [df3.reach.sum(), df3.watch_time.sum()]).quantile(
    np.concatenate((np.arange(0, 0.2, 0.05), np.arange(0.2, 1, 0.1)))
).reset_index().style.format({'index':'{:.0%}', 'reach': '{:.4%}', 'watch_time': '{:.4%}'}).hide_index()

index,reach,watch_time
0%,3.5453%,4.8338%
5%,60.2736%,62.9476%
10%,76.6284%,79.8553%
15%,84.4594%,87.0475%
20%,89.3338%,91.3241%
30%,94.8811%,95.9216%
40%,97.6082%,98.1246%
50%,98.9019%,99.1589%
60%,99.5364%,99.6613%
70%,99.8430%,99.8926%


In [107]:
df2[df2.cd.map(str) == '2022-11-15'].groupby('is_cricket').sum()

,watch_time,reach
is_cricket,,
False,1.188159e+10,5341366
True,1.702150e+08,209650


In [108]:
df2[df2.cd.map(str) == '2022-10-23'].groupby('is_cricket').sum()

,watch_time,reach
is_cricket,,
False,1.037959e+10,6526967
True,3.134469e+10,14227200


In [109]:
df2[df2.cd.map(str) == '2021-10-17'].groupby('is_cricket').sum()

,watch_time,reach
is_cricket,,
False,9.829061e+09,4800781
True,1.998365e+09,1696744


In [110]:
df4 = pd.concat([
    df3.reach.cumsum().rename('accum%') / df3.reach.sum(),
    df3.drop(['watch_time', 'reach',], axis=1),
    (df3.reach / df3.reach.sum()).rename('reach%'),
    (df3.watch_time / df3.watch_time.sum()).rename('watch_time%'),
], axis=1)
df4.head(10).style.format({'reach%': '{:.2%}', 'watch_time%': '{:.2%}', 'accum%': '{:.2%}'}).hide_index()

accum%,platform,device,language,city,state,age,gender,reach%,watch_time%
3.55%,mobile,other,english,delhi,other,other,male,3.55%,4.83%
6.47%,mobile,other,hindi,delhi,other,other,male,2.93%,2.19%
9.03%,mobile,other,english,mumbai,Maharashtra,other,male,2.55%,3.07%
11.48%,mobile,other,english,patna,Bihar + Jharkhand,other,male,2.45%,3.07%
13.84%,mobile,other,english,lucknow,Uttar Pradesh + Uttarakhand,other,male,2.36%,3.03%
16.03%,mobile,other,hindi,lucknow,Uttar Pradesh + Uttarakhand,other,male,2.19%,1.65%
18.19%,mobile,other,hindi,mumbai,Maharashtra,other,male,2.15%,1.55%
20.34%,mobile,other,hindi,patna,Bihar + Jharkhand,other,male,2.15%,1.55%
22.47%,mobile,other,english,hyderabad,Andhra Pradesh + Telangana,other,male,2.14%,2.59%
24.26%,mobile,other,english,bangalore,Karnataka,other,male,1.79%,2.29%


In [111]:
gec_days = ['2021-11-09', '2021-11-12', '2021-11-13', '2022-11-07', '2022-11-08', '2022-11-11', '2022-11-12', '2022-11-15']
df7 = df2[df2.is_cricket & ~df2.cd.map(str).isin(gec_days)].groupby(['cd'] + basic).sum().reset_index()
df7['watch_time%'] = df7.watch_time / df7.groupby('cd').watch_time.transform('sum')
df7['reach%'] = df7.reach / df7.groupby('cd').reach.transform('sum')
df7

,cd,platform,device,language,city,state,age,gender,watch_time,reach,watch_time%,reach%
0,2021-10-17,mobile,25K+,english,agra,Uttar Pradesh + Uttarakhand,30+,male,4.441338e+03,6,2.222486e-06,3.536185e-06
1,2021-10-17,mobile,25K+,english,agra,Uttar Pradesh + Uttarakhand,other,female,2.660401e+02,1,1.331289e-07,5.893641e-07
2,2021-10-17,mobile,25K+,english,agra,Uttar Pradesh + Uttarakhand,other,male,1.386226e+04,17,6.936802e-06,1.001919e-05
3,2021-10-17,mobile,25K+,english,ahmedabad,Gujarat + Goa,30+,female,1.839644e+04,30,9.205745e-06,1.768092e-05
4,2021-10-17,mobile,25K+,english,ahmedabad,Gujarat + Goa,30+,male,2.477912e+05,259,1.239969e-04,1.526453e-04
...,...,...,...,...,...,...,...,...,...,...,...,...
40590,2022-11-13,mobile,other,hindi,vijayawada,Andhra Pradesh + Telangana,other,male,6.980056e+05,920,3.256432e-05,8.601621e-05
40591,2022-11-13,mobile,other,hindi,visakhapatnam,Gujarat + Goa,30+,female,4.662875e+04,98,2.175388e-06,9.162596e-06
40592,2022-11-13,mobile,other,hindi,visakhapatnam,Gujarat + Goa,30+,male,5.930625e+05,801,2.766837e-05,7.489020e-05
40593,2022-11-13,mobile,other,hindi,visakhapatnam,Gujarat + Goa,other,female,1.823844e+05,285,8.508850e-06,2.664632e-05


In [112]:
df8 = df4.merge(df7, on=basic, suffixes=('', '_daily'))
df8

,accum%,platform,device,language,city,state,age,gender,reach%,watch_time%,cd,watch_time,reach,watch_time%_daily,reach%_daily
0,0.035453,mobile,other,english,delhi,other,other,male,3.545272e-02,4.833839e-02,2021-10-17,4.222173e+06,7921,2.112813e-03,4.668353e-03
1,0.035453,mobile,other,english,delhi,other,other,male,3.545272e-02,4.833839e-02,2021-10-18,7.596608e+07,76296,8.627284e-03,1.475785e-02
2,0.035453,mobile,other,english,delhi,other,other,male,3.545272e-02,4.833839e-02,2021-10-19,3.122954e+07,31443,7.557593e-03,1.126231e-02
3,0.035453,mobile,other,english,delhi,other,other,male,3.545272e-02,4.833839e-02,2021-10-20,1.092031e+08,87049,1.514439e-02,1.738093e-02
4,0.035453,mobile,other,english,delhi,other,other,male,3.545272e-02,4.833839e-02,2021-10-21,3.978715e+07,34274,1.582896e-02,1.620584e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40586,1.000000,mobile,other,english,gwalior,Madhya Pradesh,30+,male,1.148268e-08,5.034195e-09,2022-10-22,1.331192e+03,1,1.856950e-07,2.325309e-07
40587,1.000000,mobile,other,english,gwalior,Madhya Pradesh,30+,female,1.148268e-08,5.900975e-09,2022-10-23,1.001588e+03,1,3.195398e-08,7.028790e-08
40588,1.000000,mobile,other,english,gwalior,Madhya Pradesh,30+,female,1.148268e-08,5.900975e-09,2022-10-25,8.910880e+02,1,1.454353e-07,2.283532e-07
40589,1.000000,mobile,other,hindi,gwalior,Madhya Pradesh,30+,female,5.741341e-09,8.853065e-10,2022-10-23,2.839527e+02,1,9.059036e-09,7.028790e-08


In [113]:
df9 = df8.pivot(df4.columns, 'cd', 'reach%_daily').reset_index().fillna(0)
df9.to_csv('df9.csv', index=False, float_format='{:.2%}'.format)
df9.head(10).style.format(lambda x: f'{x:.2%}' if isinstance(x, float) else x).hide_index()

accum%,platform,device,language,city,state,age,gender,reach%,watch_time%,2021-10-17,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22,2021-10-23,2021-10-24,2021-10-25,2021-10-26,2021-10-27,2021-10-28,2021-10-29,2021-10-30,2021-10-31,2021-11-01,2021-11-02,2021-11-03,2021-11-04,2021-11-05,2021-11-06,2021-11-07,2021-11-08,2021-11-10,2021-11-11,2021-11-14,2022-10-16,2022-10-17,2022-10-18,2022-10-19,2022-10-20,2022-10-21,2022-10-22,2022-10-23,2022-10-24,2022-10-25,2022-10-26,2022-10-27,2022-10-28,2022-10-29,2022-10-30,2022-10-31,2022-11-01,2022-11-02,2022-11-03,2022-11-04,2022-11-05,2022-11-06,2022-11-09,2022-11-10,2022-11-13
3.55%,mobile,other,english,delhi,other,other,male,3.55%,4.83%,0.47%,1.48%,1.13%,1.74%,1.62%,1.02%,0.35%,0.25%,0.72%,0.39%,0.57%,0.48%,0.40%,0.48%,0.34%,0.74%,0.49%,0.31%,0.73%,0.49%,0.68%,0.58%,0.60%,0.39%,0.51%,0.25%,1.82%,3.90%,1.62%,4.91%,2.01%,1.67%,3.94%,3.19%,2.49%,3.49%,3.45%,3.33%,3.06%,3.64%,3.76%,3.65%,4.00%,3.49%,3.91%,3.88%,4.13%,3.74%,4.03%,3.57%,4.20%
6.47%,mobile,other,hindi,delhi,other,other,male,2.93%,2.19%,4.87%,3.78%,4.38%,3.48%,3.97%,4.54%,4.50%,3.99%,4.89%,4.55%,4.83%,4.82%,4.63%,5.04%,4.66%,5.34%,5.64%,4.63%,4.97%,4.48%,4.84%,5.23%,4.97%,5.07%,4.78%,4.79%,4.94%,3.08%,5.25%,1.94%,4.80%,5.50%,2.66%,2.60%,3.39%,3.14%,3.35%,2.82%,3.35%,3.09%,2.57%,3.58%,2.99%,2.49%,2.94%,2.96%,2.65%,2.66%,2.70%,2.51%,2.72%
9.03%,mobile,other,english,mumbai,Maharashtra,other,male,2.55%,3.07%,0.30%,0.76%,0.56%,0.91%,0.77%,0.54%,0.28%,0.19%,0.42%,0.30%,0.39%,0.34%,0.32%,0.37%,0.27%,0.48%,0.39%,0.27%,0.51%,0.40%,0.49%,0.45%,0.43%,0.34%,0.42%,0.24%,1.36%,2.62%,1.15%,3.36%,1.48%,1.16%,2.70%,2.45%,1.90%,2.38%,2.42%,2.48%,2.28%,2.58%,2.74%,2.45%,2.70%,2.66%,2.63%,2.71%,2.92%,2.72%,2.81%,2.69%,2.88%
11.48%,mobile,other,english,patna,Bihar + Jharkhand,other,male,2.45%,3.07%,0.37%,0.96%,0.98%,1.23%,1.35%,0.76%,0.23%,0.15%,0.53%,0.24%,0.39%,0.32%,0.25%,0.31%,0.22%,0.54%,0.31%,0.20%,0.60%,0.35%,0.57%,0.39%,0.47%,0.28%,0.30%,0.16%,1.28%,2.72%,1.17%,3.47%,1.53%,0.99%,2.72%,2.28%,1.77%,2.35%,2.40%,2.37%,2.25%,2.48%,2.57%,2.46%,2.78%,2.52%,2.63%,2.73%,2.86%,2.61%,2.66%,2.42%,2.76%
13.84%,mobile,other,english,lucknow,Uttar Pradesh + Uttarakhand,other,male,2.36%,3.03%,0.39%,1.10%,1.19%,1.37%,1.66%,0.98%,0.28%,0.18%,0.61%,0.31%,0.49%,0.42%,0.32%,0.41%,0.27%,0.67%,0.41%,0.26%,0.70%,0.43%,0.68%,0.50%,0.53%,0.32%,0.43%,0.18%,1.20%,2.56%,1.07%,3.28%,1.49%,0.98%,2.59%,2.19%,1.61%,2.32%,2.33%,2.31%,2.10%,2.47%,2.48%,2.45%,2.65%,2.42%,2.50%,2.59%,2.73%,2.50%,2.58%,2.35%,2.63%
16.03%,mobile,other,hindi,lucknow,Uttar Pradesh + Uttarakhand,other,male,2.19%,1.65%,4.10%,3.20%,4.29%,3.09%,3.79%,4.05%,3.75%,3.04%,4.31%,3.56%,4.08%,3.95%,3.64%,4.15%,3.75%,4.56%,4.53%,3.91%,4.68%,3.93%,4.47%,4.45%,4.34%,4.05%,3.90%,3.93%,3.06%,2.27%,3.68%,1.52%,3.30%,3.64%,1.95%,2.00%,2.90%,2.46%,2.70%,2.15%,3.08%,2.37%,1.83%,2.88%,2.22%,1.89%,2.23%,2.26%,2.01%,1.95%,1.88%,1.71%,1.84%
18.19%,mobile,other,hindi,mumbai,Maharashtra,other,male,2.15%,1.55%,2.33%,1.94%,2.11%,1.82%,1.88%,2.32%,2.45%,2.09%,2.40%,2.54%,2.79%,2.80%,2.80%,2.90%,2.76%,3.03%,3.40%,2.88%,2.91%,2.82%,2.87%,3.11%,2.93%,3.14%,2.96%,3.00%,3.18%,2.25%,3.56%,1.45%,3.18%,3.47%,1.90%,2.05%,2.76%,2.32%,2.48%,2.19%,2.68%,2.28%,1.80%,2.54%,2.07%,1.98%,2.10%,2.15%,1.90%,1.98%,1.92%,1.84%,1.89%
20.34%,mobile,other,hindi,patna,Bihar + Jharkhand,other,male,2.15%,1.55%,3.82%,2.86%,3.83%,2.85%,3.40%,3.56%,3.26%,2.68%,3.75%,3.06%,3.52%,3.39%,3.15%,3.48%,3.27%,3.88%,3.78%,3.35%,3.97%,3.38%,3.79%,3.74%,3.79%,3.39%,3.09%,3.41%,3.30%,2.26%,3.77%,1.51%,3.32%,3.69%,1.88%,2.00%,2.93%,2.37%,2.54%,2.11%,2.97%,2.26%,1.79%,2.73%,2.16%,1.84%,2.20%,2.20%,1.92%,1.91%,1.84%,1.68%,1.85%
22.47%,mobile,other,english,hyderabad,Andhra Pradesh + Telangana,other,male,2.14%,2.59%,1.00%,1.50%,1.28%,1.56%,1.33%,0.99%,0.98%,0.62%,1.05%,0.80%,0.92%,0.84%,0.93%,1.04%,0.76%,1.10%,0.81%,0.80%,1.32%,1.06%,1.30%,1.13%,0.87%,0.86%,1.01%,0.75%,1.74%,2.51%,1.38%,2.69%,1.34%,1.33%,2.50%,2.24%,2.25%,2.26%,2.12%,2.11%,2.00%,2.09%,2.16%,2.10%

In [115]:
topn = int(len(df3) * 0.2)
len(df3), topn

(817, 163)

In [116]:
wc2022 = df3[basic].iloc[:topn]
df3b = df2[(df2.cd.map(str) < '2022-01-01') & df2.is_cricket].groupby(basic).sum().reset_index().sort_values('reach', ascending=False)
wc2021 = df3b[basic].iloc[:topn]

In [117]:
a = set(wc2022.apply(tuple, axis=1))
b = set(wc2021.apply(tuple, axis=1))
len(a), len(b), len(a&b)

(163, 163, 110)

In [119]:
df3c = df3b[df3b[basic].apply(lambda x:tuple(x) in b-a, axis=1)].copy()
df3c.watch_time = df3c.watch_time / df3b.watch_time.sum()
df3c.reach = df3c.reach / df3b.reach.sum()
print(df3c[['watch_time', 'reach']].sum())
df3c

watch_time    0.112120
reach         0.120996
dtype: float64


,platform,device,language,city,state,age,gender,watch_time,reach
634,mobile,other,hindi,bhopal,Madhya Pradesh,other,male,0.007015,0.006791
752,mobile,other,hindi,other,Maharashtra,other,female,0.004029,0.005192
664,mobile,other,hindi,hyderabad,Andhra Pradesh + Telangana,other,female,0.003857,0.004787
784,mobile,other,hindi,pune,Maharashtra,other,female,0.003566,0.004612
717,mobile,other,hindi,other,Andhra Pradesh + Telangana,other,male,0.003951,0.004396
741,mobile,other,hindi,other,Karnataka,other,male,0.003994,0.004193
736,mobile,other,hindi,other,Haryana + Punjab + Himachal Pradesh + Jammu an...,other,female,0.003011,0.004064
260,mobile,25K+,hindi,hyderabad,Andhra Pradesh + Telangana,other,male,0.003952,0.003672
646,mobile,other,hindi,coimbatore,Tamil Nadu + Pondicherry,other,male,0.003453,0.003506
765,mobile,other,hindi,other,Tamil Nadu + Pondicherry,other,male,0.003051,0.003404


In [120]:
df3d = df2[(df2.cd.map(str) > '2022-01-01') & df2.is_cricket].groupby(basic).sum().reset_index().sort_values('reach', ascending=False)
df3e = df3d[df3d[basic].apply(lambda x:tuple(x) in a-b, axis=1)].copy()
df3e.watch_time = df3e.watch_time / df3d.watch_time.sum()
df3e.reach = df3e.reach / df3d.reach.sum()
print(df3e[['watch_time', 'reach']].sum())

watch_time    0.154381
reach         0.130806
dtype: float64


In [122]:
gec_days = ['2021-11-09', '2021-11-12', '2021-11-13', '2022-11-07', '2022-11-08', '2022-11-11', '2022-11-12', '2022-11-15']
df7 = df2[~df2.is_cricket&df2.cd.map(str).isin(gec_days)].groupby(['cd'] + basic).sum().reset_index()
df7['watch_time%'] = df7.watch_time / df7.groupby('cd').watch_time.transform('sum')
df7['reach%'] = df7.reach / df7.groupby('cd').reach.transform('sum')
df8 = df4.merge(df7, on=basic, suffixes=('', '_daily'))
df9 = df8.pivot(df4.columns, 'cd', 'reach%_daily').reset_index().fillna(0)
df9.to_csv('df9gec.csv', index=False, float_format='{:.2%}'.format)